In [1]:
import numpy as np
import os
import random
import tensorflow as tf

from tensorflow.python.platform import flags
from utils import get_images

FLAGS = flags.FLAGS

In [5]:
batch_size = batch_size
num_samples_per_class = num_samples_per_class
num_classes = config.get('num_classes', FLAGS.num_classes)
img_size = (84, 84)          
dim_input = np.prod(self.img_size)*3
dim_output = self.num_classes
metatrain_folder = config.get('metatrain_folder', './mini-imagenet/train')
if FLAGS.test_set:
    metaval_folder = config.get('metaval_folder', './mini-imagenet/test')
else:
    metaval_folder = config.get('metaval_folder', './mini-imagenet/val')

metatrain_folders = [os.path.join(metatrain_folder, label) \
                for label in os.listdir(metatrain_folder) \
                if os.path.isdir(os.path.join(metatrain_folder, label)) \
                ]
metaval_folders = [os.path.join(metaval_folder, label) \
                for label in os.listdir(metaval_folder) \
                if os.path.isdir(os.path.join(metaval_folder, label)) \
                ]
metatrain_character_folders = metatrain_folders
metaval_character_folders = metaval_folders
rotations =  [0]

if train:
            folders = self.metatrain_character_folders
            # number of tasks, not number of meta-iterations. (divide by metabatch size to measure)
            num_total_batches = 200000
else:
            folders = self.metaval_character_folders
            num_total_batches = 600

# make list of files
print('Generating filenames')
all_filenames = []
for _ in range(num_total_batches):
            sampled_character_folders = random.sample(folders, self.num_classes)
            random.shuffle(sampled_character_folders)
            labels_and_images = get_images(sampled_character_folders, range(self.num_classes), nb_samples=self.num_samples_per_class, shuffle=False)
            # make sure the above isn't randomized order
            labels = [li[0] for li in labels_and_images]
            filenames = [li[1] for li in labels_and_images]
            all_filenames.extend(filenames)

        

       

        
        # make queue for tensorflow to read from
        #filename_queue = tf.train.string_input_producer(tf.convert_to_tensor(all_filenames), shuffle=False)       
        
        print('Generating image processing ops')
        #image_reader = tf.WholeFileReader()
        #To read filenames from the queue, you can use the dequeue() method:
        
        #This code will dequeue the next filename from the queue and return it as a tensor. You can then use this filename to read the corresponding image file.
       
        filename = filename_queue.dequeue()        
        #how to read an image file from the filename
        image_bytes = tf.io.read_file(filename)
        #_, image_file = image_reader.read(filename_queue)
        image = tf.image.decode_jpeg(image_bytes, channels=3)
        image.set_shape((self.img_size[0],self.img_size[1],3))
        image = tf.reshape(image, [self.dim_input])
        image = tf.cast(image, tf.float32) / 255.0
        num_preprocess_threads = 1 # TODO - enable this to be set to >1
        min_queue_examples = 256
        examples_per_batch = self.num_classes * self.num_samples_per_class
        batch_image_size = self.batch_size  * examples_per_batch
        print('Batching images')
        images = tf.train.batch(
                [image],
                batch_size = batch_image_size,
                num_threads=num_preprocess_threads,
                capacity=min_queue_examples + 3 * batch_image_size,
                )
        all_image_batches, all_label_batches = [], []
        print('Manipulating image data to be right shape')
        for i in range(self.batch_size):
            image_batch = images[i*examples_per_batch:(i+1)*examples_per_batch]

            label_batch = tf.convert_to_tensor(labels)
            new_list, new_label_list = [], []
            for k in range(self.num_samples_per_class):
                class_idxs = tf.range(0, self.num_classes)
                class_idxs = tf.random_shuffle(class_idxs)

                true_idxs = class_idxs*self.num_samples_per_class + k
                new_list.append(tf.gather(image_batch,true_idxs))
                new_label_list.append(tf.gather(label_batch, true_idxs))
            new_list = tf.concat(new_list, 0)  # has shape [self.num_classes*self.num_samples_per_class, self.dim_input]
            new_label_list = tf.concat(new_label_list, 0)
            all_image_batches.append(new_list)
            all_label_batches.append(new_label_list)
        all_image_batches = tf.stack(all_image_batches)
        all_label_batches = tf.stack(all_label_batches)
        all_label_batches = tf.one_hot(all_label_batches, self.num_classes)
        return all_image_batches, all_label_batches        

In [10]:
if FLAGS.train:
   data_generator = DataGenerator(5, 1)

UnrecognizedFlagError: Unknown command line flag 'f'